# Day 8: Advanced Optimization with Mosek
## Stochastic Portfolio Optimization with SOCP
This notebook implements a stochastic optimization model using Mosek's SOCP capabilities for robust portfolio construction.

In [ ]:
from mosek.fusion import *
import numpy as np

def stochastic_optimization(returns_scenarios: list, probabilities: np.ndarray, target_return: float) -> np.ndarray:
    n = returns_scenarios[0].shape[1]
    with Model("StochasticOpt") as M:
        w = M.variable("w", n, Domain.unbounded())
        t = M.variable("t", 1, Domain.unbounded())

        # Objective: Minimize weighted sum of scenario variances
        obj = Expr.add([prob * Expr.dot(w, Expr.mul(Sigma, w)) for prob, Sigma in zip(probabilities, [np.cov(scenario.T) for scenario in returns_scenarios])])
        M.objective("obj", ObjectiveSense.Minimize, obj)

        # Constraints: Target return per scenario
        for scenario in returns_scenarios:
            M.constraint(Expr.dot(scenario.mean(axis=0), w), Domain.greaterThan(target_return))

        M.solve()
        return w.level()

# Example usage
np.random.seed(42)
scenarios = [pd.DataFrame(np.random.normal(0, 0.01, (100, 3))), pd.DataFrame(np.random.normal(0, 0.02, (100, 3)))]
probabilities = np.array([0.7, 0.3])
target_return = 0.05
optimal_weights = stochastic_optimization(scenarios, probabilities, target_return)
print(f'Optimal Weights: {optimal_weights}')

## Notes
- **Stochastic Programming**: Ensures robustness across multiple market scenarios.
- **SOCP**: Efficiently handles non-linear constraints like variance.
- **Transaction Costs**: Can be incorporated using piecewise linear approximations.